In [273]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

Load and merged dataset

In [301]:
file_paths = ["../../data/Stats_csv/Processed_Forward_data.csv",  
"../../data/Stats_csv/Processed_Midfielder_data.csv"]
dfs = []
for file in file_paths:
    dfs.append(pd.read_csv(file))
common_columns = set(dfs[0].columns) 
for df in dfs[1:]:
    common_columns &= set(df.columns) 

common_columns = list(common_columns)
merged_df = []
for df in dfs:
    merged_df.append(df[common_columns])

merged_df = pd.concat(merged_df,ignore_index=True)
merged_df = merged_df.loc[merged_df['Appearances']!=0]


Data cleaning

In [302]:
from sklearn.preprocessing import LabelEncoder

for col in ['Position', 'Nationality', 'Club']:
    merged_df[col] = LabelEncoder().fit_transform(merged_df[col])
merged_df.columns

Index(['Goals with right foot', 'Crosses', 'Height', 'Penalties scored',
       'Shooting accuracy %', 'Big chances missed', 'Yellow cards',
       'Blocked shots', 'Assists', 'Nationality', 'Freekicks scored', 'Shots',
       'Offsides', 'Headed Clearance', 'Appearances', 'Position',
       'Interceptions', 'Headed goals', 'Goals with left foot', 'Passes',
       'Clearances', 'Hit woodwork', 'Tackles', 'Red cards', 'Date of Birth',
       'Name', 'Goals', 'Losses', 'Club', 'Big Chances Created', 'Fouls',
       'Wins', 'Goals per match', 'Passes per match', 'Shots on target'],
      dtype='object')

In [328]:
features = ['Appearances', 'Assists', 'Fouls', 'Passes', 'Wins', 
            'Passes per match', 'Height', 'Red cards', 'Yellow cards', 'Losses','Position','Nationality','Club']
target = 'Goals'

X = merged_df[features]
y = merged_df[target]

Split data

In [329]:
X_train, X_test , y_train , y_test = train_test_split(X,y,test_size=0.3, random_state=42)

In [330]:
scaler = StandardScaler()
X_train_scaler = scaler.fit_transform(X_train)
X_test_scaler = scaler.transform(X_test)

In [331]:
model = LogisticRegression(random_state=42, max_iter=500)
model.fit(X_train_scaler, y_train)
y_pred = model.predict(X_test_scaler)
accuracy = accuracy_score(y_test, y_pred)
print("Prediction Accuracy (Logistic Regression):", accuracy)
print(classification_report(y_test, y_pred))

Prediction Accuracy (Logistic Regression): 0.5333333333333333
              precision    recall  f1-score   support

           0       0.67      0.87      0.76        39
           1       0.23      0.20      0.21        15
           2       0.20      0.11      0.14         9
           3       0.00      0.00      0.00         7
           4       0.50      0.67      0.57         3
           5       0.00      0.00      0.00         2
           7       0.00      0.00      0.00         0

    accuracy                           0.53        75
   macro avg       0.23      0.26      0.24        75
weighted avg       0.44      0.53      0.48        75



c:\Users\hoang\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\hoang\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\hoang\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 

In [ ]:
# Dự đoán xác suất cho tập kiểm tra
X_scaled = scaler.transform(X)
predictions = model.predict_proba(X_scaled)[:, 0]
merged_df['Top_scorer'] = predictions
x = merged_df[['Name','Top_scorer']].sort_values(by='Top_scorer', ascending=False)
x


In [333]:
X_scaled = scaler.transform(X)  # Chuẩn hóa toàn bộ dữ liệu X
predictions = model.predict_proba(X_scaled)[:, 0]  # Xác suất của lớp 1
merged_df['Top_scorer'] = predictions

# Kiểm tra và sắp xếp kết quả
if 'Name' in merged_df.columns:
    result = merged_df[['Name', 'Top_scorer']].sort_values(by='Top_scorer', ascending=True)
else:
    result = merged_df[['Top_scorer']].sort_values(by='Top_scorer', ascending=True)

print(result.head())


             Name  Top_scorer
3       Joelinton    0.009897
23  Mohamed Salah    0.011773
36    Bukayo Saka    0.013399
25      Luis Díaz    0.019510
38      Evanilson    0.024033
